In [1]:
import pandas as pd
import evaluate
import re
import sqlite3
import random
from tqdm import tqdm
import sys
import numpy as np

random.seed(10001)

bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pd.set_option('display.max_columns', 101) #replace n with the number of columns you want to see completely
# pd.set_option('display.max_rows', 101) #replace n with the number of rows you want to see completely

In [3]:
df = pd.read_csv("generated_data/gpt4/gpt4_dynamic_few_shot_6_mod_50.csv")
sqlite_path = "generated_data/testing/accounting_for_testing.sqlite"

test_size = len(df)
print("Total test size: ", test_size)
# print(df.head())
# print(df.columns)

pred_score = df['PREDICTED SQL'].str.lower().values
# debug_score = df['DEBUGGED SQL'].str.lower().values
gold_score1 = df['GOLD SQL'].str.lower().values

Total test size:  401


In [5]:
def replace_current_date_and_now(_sql, _date):
    _sql = _sql.replace('current_date', "\'"+_date+"\'")
    _sql = _sql.replace(', now', ", \'"+_date+"\'")
    return _sql

def replace_percent_symbol_y(_sql):
    _sql = _sql.replace('%y', "%Y")
    return _sql


In [27]:
double_entry = []

def remove_gold_Non_exec(df1, sqlite_path):

    con = sqlite3.connect(sqlite_path)
    cur = con.cursor()

    out, non_exec=[], []
    new_df = df1.copy()
    new_df.loc[:, 'Exec/Non-Exec'] = 0
    new_df.loc[:, 'GOLD_res'] = None
    responses=[]

    f = 0
    for i,s in tqdm(enumerate(new_df["GOLD SQL"].values)):
        _sql = str(s).replace('"', "'").lower()
        _sql = replace_current_date_and_now(_sql, '2022-06-01')
        _sql = replace_percent_symbol_y(_sql)
        try:
            cur.execute(_sql)
            res = cur.fetchall()
            responses.append({i: res})
        except:
            print("sql: ", _sql)
            non_exec.append(i)
        

    idx = [list(x.keys())[0] for x in responses]
    val = [list(x.values())[0] for x in responses]
    new_df.loc[idx, 'Exec/Non-Exec'] = 1
    new_df.loc[idx, 'GOLD_res'] = val

    con.close()    
    out=idx

    return out, non_exec, new_df

In [28]:
print("Checking non-exec Gold sql query")
gold_exec, gold_not_exec, new_df = remove_gold_Non_exec( df, sqlite_path)
print("GOLD Total exec SQL query: {}/{}".format(len(gold_exec), test_size))
print("GOLD Total non-exec SQL query: {}/{}".format(len(gold_not_exec), test_size))

Checking non-exec Gold sql query


401it [01:38,  4.09it/s]


GOLD Total exec SQL query: 401/401
GOLD Total non-exec SQL query: 0/401


In [34]:
new_df.iloc[236][4]

"[('maintenance and repair', 766151.7300000001)]"

In [32]:
q = new_df.iloc[236,2]
# q = df.iloc[0,1]
q = str(q).replace('"', "'").lower()
q = replace_current_date_and_now(q, '2022-06-01')
q = replace_percent_symbol_y(q)

con = sqlite3.connect(sqlite_path)
cur = con.cursor()

cur.execute(q)
res = cur.fetchall()

print(res)




[('maintenance and repair', 766151.7300000001)]


In [33]:
res

[('maintenance and repair', 766151.7300000001)]

In [25]:
q

"select account, sum(debit) from master_txn_table  as t1 join chart_of_accounts as t2 on t1.account = t2.account_name where account_type in ('expenses','other expenses') and transaction_date between date('2022-06-01', '-1 year', 'start of year', '+6 month') and date('2022-06-01', '-1 year', 'start of year', '+9 month', '-1 day')  order by sum(debit) desc limit 1"

In [38]:
q

'select distinct transaction_id, AR_paid from master_txn_table where product_service = "Recreational programs" and transaction_date BETWEEN date( \'2022-06-01\', "-1 day") AND date( \'2022-06-01\', "-1 day") '

In [39]:
t = new_df.loc[:20,:].copy()
ls = [0,12]
t.loc[ls,"GOLD_res"] = "asdf"


In [40]:
t

,NLQ,PREDICTED SQL,GOLD SQL,Exec/Non-Exec,GOLD_res
0,Display all transactions involving Nathan Carter,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,asdf
1,Display all transactions involving Garrett Pet...,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(6710,)"
2,Display all transactions involving Nancy Holloway,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(21907,)"
3,Display all transactions involving Kimberly Bi...,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(6077,)"
4,Display all transactions involving Matthew Mat...,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(28179,)"
5,Display all transactions involving Michelle Go...,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(4626,)"
6,Display all transactions involving Tina Willis,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(5477,)"
7,Display all transactions involving Nicole Roberts,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(36400,)"
8,Display all transactions involving Brenda Cobb,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(41982,)"
9,Display all transactions involving Jason Riley,SELECT distinct transaction_id FROM master_txn...,select distinct transaction_id from master_txn...,1,"(7006,)"


In [13]:
new_df.iloc[236]

NLQ                What was our greatest expenses in q3 last year?
PREDICTED SQL    SELECT account, sum(debit) FROM master_txn_tab...
GOLD SQL         select account, sum(debit) from master_txn_tab...
Exec/Non-Exec                                                    1
GOLD_res             ('maintenance and repair', 766151.7300000001)
Name: 236, dtype: object

In [2]:
import sqlite3
def update_task(conn, task):

    """
    update priority, begin_date, and end date of a task
    :param conn:
    :param task:
    :return: project id
    """
    sql = ''' UPDATE master_txn_table
              SET transaction_date = ?
              WHERE transaction_date = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

database = r"generated_data/testing/updated_accounting_for_testing.sqlite"
# create a database connection
conn = sqlite3.connect(database)
with conn:
    update_task(conn, ('2021-07-01', '2022-07-01'))